In [1]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import numpy as np
from time import time


In [2]:
num_arrays = 200 # We will create 200 random arrays

array_len = 1024**2 # the length of the arrays


In [3]:
ker = SourceModule("""       
__global__ void mult_ker(float * array, int array_len)
{
     
     // we will use one block with blockDim.x=64 number of threads.
     
     // we will need num_iters number of iterations to process one array
     int num_iters = array_len / blockDim.x;

     // current thread id
     int thd = blockIdx.x*blockDim.x + threadIdx.x; 
     
     // iterate over different parts of the array
     for(int j=0; j < num_iters; j++)
     {
         int i = j * blockDim.x + thd;

         for(int k = 0; k < 50; k++)
         {
              array[i] *= 2.0;
              array[i] /= 2.0;
         }
     }

}
""")

mult_ker = ker.get_function('mult_ker')


In [4]:
data = []
data_gpu = []
gpu_out = []


In [5]:
# generate random arrays.
for _ in range(num_arrays):
    data.append(np.random.randn(array_len).astype('float32'))


In [8]:
t_start = time()

# copy arrays to GPU.Serial upload
for k in range(num_arrays):
    data_gpu.append(gpuarray.to_gpu(data[k]))

# process arrays.Run num_arrays (= 200) kernel functions serially.
for k in range(num_arrays):
    mult_ker(data_gpu[k], np.int32(array_len), block=(64,1,1), grid=(1,1,1))

# copy arrays from GPU.Serial data download.
for k in range(num_arrays):
    gpu_out.append(data_gpu[k].get())

t_end = time()

print('Total time: %f' % (t_end - t_start))


Total time: 2.873603


In [9]:
for k in range(num_arrays):
    assert (np.allclose(gpu_out[k], data[k]))

